In [1]:

import pandas as pd
from datetime import datetime, timedelta

train = pd.read_csv('train.csv.zip')
train['DATE'] = pd.to_datetime(train['DATE'])

# Unique ATMs
ATM_IDs = train.ATM_ID.unique()

# The dates to predict
pred_dates  = ['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
               '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
               '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
               '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31',
               '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
               '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
               '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12',
               '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16',
               '2017-09-17']

In [2]:
valid_dates = pd.date_range('2017-07-14', '2017-08-15').tolist()
len(valid_dates)

33

In [3]:
rows = []
for ATM in ATM_IDs[:50]: 
    for i in range(len(valid_dates)):
        date = valid_dates[i]
        value = train[(train.ATM_ID==ATM) & (train.DATE==valid_dates[i])]['CLIENT_OUT'].values[0]
        rows.append([date, ATM, value])

submission_valid = pd.DataFrame(rows, columns = ['DATE', 'ATM_ID', 'CLIENT_OUT'])                    
submission_valid.shape

(1650, 3)

In [4]:
year_ago = pd.to_datetime(valid_dates) - timedelta(days=31)
# year_ago = pd.date_range('2017-03-01', '2017-04-02').tolist()
rows = []
for ATM in ATM_IDs[:50]: 
    for i in range(len(valid_dates)):
        date = valid_dates[i]
        value = train[(train.ATM_ID==ATM) & (train.DATE==year_ago[i])]['CLIENT_OUT'].values[0]
        rows.append([date, ATM, value])

submission = pd.DataFrame(rows, columns = ['DATE', 'ATM_ID', 'CLIENT_OUT'])   

In [5]:
from score_submission import scorer, check_submission
check_submission(submission)
scorer(submission_valid, submission)*10000

9978.459669880152

In [9]:
%matplotlib inline
from matplotlib import pyplot as plt 

# plot figures and seve to folder
for atm in ATM_IDs[:50]:
    fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
#     ax.plot(submission_valid[submission_valid.ATM_ID==atm].CLIENT_OUT)
    ax.plot(train[(train.ATM_ID==atm) & (train.DATE>valid_dates[0] - timedelta(days=61))].CLIENT_OUT)
    fig.savefig('pics/'+str(atm)+'.png')   # save the figure to file
    # plt.show()
    plt.close(fig)    # close the figure

    

In [83]:
def replace_week_ago(trainnine):
    fist_q =  trainnine.CLIENT_OUT.quantile(0.9)
    last_q = trainnine.CLIENT_OUT.quantile(0.1)
    mask = (trainnine.CLIENT_OUT > fist_q) | (trainnine.CLIENT_OUT < last_q) 
    trainnine['mask'] = mask
    trainnine['shifted'] = trainnine.CLIENT_OUT.shift(7)
    trainnine.loc[mask,['CLIENT_OUT','shifted', ]] = trainnine.loc[mask,['shifted', 'CLIENT_OUT']].values
    trainnine = trainnine.iloc[7:]
    return trainnine[['DATE', "ATM_ID", 'CLIENT_OUT']]


def apply_replace(train):
    train2 = train.iloc[:0].copy()
    ATM_IDs = train.ATM_ID.unique()
    for ATM in ATM_IDs:
        trainnine = train[(train.ATM_ID==ATM)].copy()
        trainnine = replace_week_ago(trainnine)
    #     print(trainnine.head())
        train2 = pd.concat([train2, trainnine])
    train = train2.copy()
    del train2
    return train

In [76]:
trainnine = train[(train.ATM_ID==9)].copy()
def replace_week_ago(trainnine):
    fist_q =  trainnine.CLIENT_OUT.quantile(0.9)
    last_q = trainnine.CLIENT_OUT.quantile(0.1)
    mask = (trainnine.CLIENT_OUT > fist_q) | (trainnine.CLIENT_OUT < last_q) 
    trainnine['mask'] = mask
    trainnine['shifted'] = trainnine.CLIENT_OUT.shift(7)
    trainnine.loc[mask,['CLIENT_OUT','shifted', ]] = trainnine.loc[mask,['shifted', 'CLIENT_OUT']].values
    trainnine = trainnine.iloc[7:]
    return trainnine[['DATE', "ATM_ID", 'CLIENT_OUT']]
trainnine = replace_week_ago(trainnine)
trainnine

,DATE,ATM_ID,CLIENT_OUT
7678,2015-01-15,9,233800.0
7679,2015-01-16,9,140600.0
7680,2015-01-17,9,48800.0
7681,2015-01-18,9,52500.0
7682,2015-01-19,9,190100.0
7683,2015-01-20,9,153700.0
7684,2015-01-21,9,230300.0
7685,2015-01-22,9,136400.0
7686,2015-01-23,9,235100.0
7687,2015-01-24,9,51200.0
